# ETL Process
### Extract, Transform, Load

In [1]:
# Importing relevant packages

import requests
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

### Extract

We gathered our data from the 2019 Annual Business Survey (ABS) through their [APIs](https://www.census.gov/data/developers/data-sets/abs.2019.html).

In order to pull data from the API, you need to use an API key, which can be requested from the [API User Guide](https://www.census.gov/data/developers/guidance/api-user-guide.Help_&_Contact_Us.html).

With our API key, we generate urls to the various datasets including all the parameters of interest in our get variables. We probably won't use all of this information but it's better to have it available and trim it down later.

In [2]:
API_KEY = '2e3a785a0323fb22d89acae1264b47cb449782aa'

#Setting up Url for API calls
get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F'
summary_us_url = f'https://api.census.gov/data/2018/abscs?get={get}&for=us:*&key={API_KEY}'
summary_state_url = f'https://api.census.gov/data/2018/abscs?get={get}&for=state:*&key={API_KEY}'

get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F'
business_char_workers_us_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=us:*&QDESC_LABEL=WORKERS&key={API_KEY}'
business_char_workers_state_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=state:*&QDESC_LABEL=WORKERS&key={API_KEY}'
business_char_cust_us_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=us:*&QDESC_LABEL=CUST&key={API_KEY}'
business_char_cust_state_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=state:*&QDESC_LABEL=CUST&key={API_KEY}'
business_char_family_us_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=us:*&QDESC_LABEL=FAMOWN&key={API_KEY}'
business_char_family_state_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=state:*&QDESC_LABEL=FAMOWN&key={API_KEY}'

get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F'
owner_educ_us_url = f'https://api.census.gov/data/2018/abscbo?get={get}&for=us:*&QDESC_LABEL=EDUC&key={API_KEY}'
owner_educ_state_url = f'https://api.census.gov/data/2018/abscbo?get={get}&for=state:*&QDESC_LABEL=EDUC&key={API_KEY}'
owner_age_us_url = f'https://api.census.gov/data/2018/abscbo?get={get}&for=us:*&QDESC_LABEL=OWNRAGE&key={API_KEY}'
owner_age_state_url = f'https://api.census.gov/data/2018/abscbo?get={get}&for=state:*&QDESC_LABEL=OWNRAGE&key={API_KEY}'

get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,NSFSZFI,NSFSZFI_LABEL,TECHUSE,TECHUSE_LABEL,FACTORS_U,FACTORS_U_LABEL,IMPACTWF_U,IMPACTWF_U_LABEL,MOTUSETECH,MOTUSETECH_LABEL,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F'
tech_us_url = f'https://api.census.gov/data/2018/abstcb?get={get}&for=us:*&key={API_KEY}'
tech_state_url = f'https://api.census.gov/data/2018/abstcb?get={get}&for=state:*&key={API_KEY}'



Once we have all the urls, we can make the requests using<br>
    
    requests.get()
Storing all the responses for use so we don't have to repeatedly call the API.<br> (API calls are limited to 500 a day)

In [3]:
# Making API calls

# summary_us_request = requests.get(summary_us_url)
# summary_state_request = requests.get(summary_state_url)
business_char_workers_us_request = requests.get(business_char_workers_us_url)
business_char_workers_state_request = requests.get(business_char_workers_state_url)
business_char_cust_us_request = requests.get(business_char_cust_us_url)
business_char_cust_state_request = requests.get(business_char_cust_state_url)
business_char_family_us_request = requests.get(business_char_family_us_url)
business_char_family_state_request = requests.get(business_char_family_state_url)
owner_educ_us_request = requests.get(owner_educ_us_url)
owner_educ_state_request = requests.get(owner_educ_state_url)
owner_age_us_request = requests.get(owner_age_us_url)
owner_age_state_request = requests.get(owner_age_state_url)
tech_us_request = requests.get(tech_us_url)
# tech_state_request = requests.get(tech_state_url)


In [28]:
text = business_char_family_us_request.text

df = pd.read_json(text)

new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

df['FIRMPDEMP'] = df['FIRMPDEMP'].astype(np.int64, errors='ignore')

df = df.groupby(['NAICS2017_LABEL', 'RACE_GROUP_LABEL'])['RCPPDEMP'].sum().unstack()

df.head(40)

RACE_GROUP_LABEL,American Indian and Alaska Native,Asian,Black or African American,Classifiable,Equally minority/nonminority,Minority,Native Hawaiian and Other Pacific Islander,Nonminority,Total,Unclassifiable,White
NAICS2017_LABEL,,,,,,,,,,,
Accommodation and food services,00000,00000,00000,00000,00000000000000000000000000000,0000000000000000000000000000000000,00000,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
Administrative and support and waste management and remediation services,0000,00000,00000,00000,000000000000000000000000000000,00000000000000000000000000000000000,0000,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
"Agriculture, forestry, fishing and hunting",00000,0000,000,00000,000000000000,0000000000000000000000000,000,000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
"Arts, entertainment, and recreation",0000,00000,0000,00000,00000000000000000000000000,0000000000000000000000000000000,NaN,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
Construction,00000,00000,00000,00000,0000000000000000000000000000,0000000000000000000000000000000000,0000,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
Educational services,0000,00000,00000,00000,000000000000000000000,00000000000000000000000000000000,000,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
Finance and insurance,00000,00000,00000,00000,0000000000000000000000000000000,0000000000000000000000000000000000,0000,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
Health care and social assistance,00000,00000,00000,00000,000000000000000000000000000000000,00000000000000000000000000000000000,00000,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000
Industries not classified,00000,00000,00000,00000,00000000000000000000,00000000000000000000000000000000,0000,00000000000000000000000000000000000,0000000000000000000000000000000000000000000000...,00,00000


In [38]:
text = business_char_workers_us_request.text

df = pd.read_json(text)

new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

df['PAYANN'] = df['PAYANN'].astype(np.int64, errors='ignore')
df['EMP'] = df['EMP'].astype(np.int64, errors='ignore')

df['foo'] = round(df['PAYANN'] / df['EMP'])

df = df.groupby(['NAICS2017_LABEL', 'RACE_GROUP_LABEL', 'BUSCHAR_LABEL']).agg(foo=('foo', 'mean')).unstack()

df.head(40)

foo  \
BUSCHAR_LABEL                                                                                  All firms   
NAICS2017_LABEL                                    RACE_GROUP_LABEL                                        
Accommodation and food services                    American Indian and Alaska Native           18.000000   
                                                   Asian                                       19.000000   
                                                   Black or African American                   16.000000   
                                                   Classifiable                                19.000000   
                                                   Equally minority/nonminority                18.200000   
                                                   Minority                                    18.666667   
                                                   Native Hawaiian and Other Pacific Islander  21.000000   
                                                   Nonminority                                 18.428571   
                                                   Total                                       18.625000   
                                                   Unclassifiable                              24.000000   
                                                   White                                       19.000000   
Administrative and support and waste management... American Indian and Alaska Native           38.000000   
                                                   Asian                                       35.000000   
                                                   Black or African American                   31.000000   
                                                   Classifiable                                35.000000   
                                                   Equally minority/nonminority                35.000000   
                                                   Minority                                    32.285714   
                                                   Native Hawaiian and Other Pacific Islander  25.000000   
                                                   Nonminority                                 34.428571   
                                                   Total                                       33.281250   
                                                   Unclassifiable                              46.000000   
                                                   White                                       35.000000   
Agriculture, forestry, fishing and hunting         American Indian and Alaska Native                 NaN   
                                                   Asian                                             NaN   
                                                   Black or African American                         NaN   
                                                   Classifiable                                17.000000   
                                                   Equally minority/nonminority                      NaN   
                                                   Minority                                    36.000000   
                                                   Native Hawaiian and Other Pacific Islander  10.000000   
                                                   Nonminority                                 21.166667   
                                                   Total                                       20.750000   
                                                   Unclassifiable                              40.000000   
                                                   White                                       20.000000   
Arts, entertainment, and recreation                American Indian and Alaska Native                 NaN   
                                                   Asian                                       40.000000 

In [93]:
df['QDESC_LABEL'].value_counts()

FAMOWN    6172
Name: QDESC_LABEL, dtype: int64

### Transform

In this step, we build a function to do some preliminary cleaning across the datasets. It takes in the stored request from the API calls to clean the data.

It accomplishes this task by:
+ Removing the brackets and quotes
+ Changing fields with extra commas
+ Splitting each entry into it's own row
+ Transforming the split string into a list
+ Extracting the headers and values
+ Creating the DataFrame

Once the DataFrame is created, we then take a few more steps to prep our data for use:
+ Changing the capitalized header names to lower case
+ Renaming columns to more descriptive names
+ Changing numercial columns from string type to numerical

In [24]:
def to_dataframe(response):
    '''Takes API response, converts it into text, cleans it and places it 
    into a pandas dataframe which is returned to the user for analysis'''

    data = response.text
    # Remove brackets from ends
    data = data[2:-2]
    # Remove quotes from values
    data = data.replace('"', '')
    # Altering values containing commas so they don't misalign and break the DataFrame
    data = data.replace('Agriculture, forestry, fishing and hunting', 'Agriculture/Forestry/Fishing/Hunting')
    data = data.replace('Arts, entertainment, and recreation', 'Arts/Entertainment/Recreation')
    data = data.replace('Mining, quarrying, and oil and gas extraction', 'Mining/Quarrying/Oil&Gas Extraction')
    data = data.replace('Not applicable,', 'N/A')
    data = data.replace('Professional, scientific, and technical services', 'Professional/Scientific/Technical Services')
    data = data.replace('Some college, but no degree', 'Some college')
    data = data.replace('Technical, trade, or vocational', 'Technical/Trade/Vocational')
    data = data.replace('Tested,','Tested')
    data = data.replace('Other businesses, including distributors of your product(s)', 'Other businesses')
    data = data.replace('State and local government, including school districts, transportation authorities, etc.', 'State and local government')
    data = data.replace('Contractors, subcontractors, independent contractors, or outside consultants', 'Contractors/subcontractors/independent contractors/outside consultants')

    # Splitting into rows
    data = data.split('],\n[')
    # Changing into a list to seperate headers and values
    new_data = [i.split(',') for i in data]

    # Creating DataFrame
    values = new_data[1:]
    headers = new_data[0]
    df = pd.DataFrame(values, columns = headers)
    # Convert names into lower case
    df.columns = df.columns.str.lower()
    df = df.replace('null', np.nan)

    # Change numeric columns from str to numeric
    numeric = [
        'emp',
        'emp_pct',
        'firmpdemp',
        'firmpdemp_pct',
        'payann',
        'payann_pct',
        'rcppdemp',
        'rcppdemp_pct',
        ]
    for i in numeric:
        # df[i] = pd.to_numeric(df[i])
        df[i] = df[i].astype(np.int64, errors='ignore')
        
    # Renaming columns to more descriptive names
    df.rename(columns = {
        'emp' : 'number_of_employees',
        'emp_pct' : 'percent_of_employees',
        'eth_group_label' : 'ethnicity',
        'factors_u_label' : 'usage_adverse_factors',
        'firmpdemp' : 'number_of_firms',
        'firmpdemp_pct' : 'percent_of_firms',
        'impactwf_u_label' : 'tech_usage_worker_impact',
        'motusetech_label' : 'tech_usage_purpose',
        'naics2017_label' : 'industry',
        'nsfszfi_label' : 'firm_size_category',
        'payann' : 'annual_payroll',
        'payann_pct' : 'percent_annual_payroll',
        'race_group_label' : 'race',
        'rcppdemp' : 'revenue',
        'rcppdemp_pct' : 'percent_revenue_of_firms',
        'sex' : 'sex_code',
        'sex_label' : 'sex',
        'techuse_label' : 'tech_usage'
    }, inplace = True)


    return df

In [ ]:
# # Chris
# tech = to_dataframe(tech_us_request)
# # tech.iloc[:,10:30].info()
# tech.head(20)

In [ ]:
# Sara

In [ ]:
# Jasleen

In [42]:
workers = business_char_workers_us_request.text

df = pd.read_json(workers)

new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header



df.columns

Index(['GEO_ID', 'NAME', 'NAICS2017', 'NAICS2017_LABEL', 'SEX', 'SEX_LABEL',
       'ETH_GROUP', 'ETH_GROUP_LABEL', 'RACE_GROUP', 'RACE_GROUP_LABEL',
       'VET_GROUP', 'VET_GROUP_LABEL', 'QDESC', 'BUSCHAR', 'BUSCHAR_LABEL',
       'YEAR', 'FIRMPDEMP', 'FIRMPDEMP_F', 'FIRMPDEMP_PCT', 'FIRMPDEMP_PCT_F',
       'RCPPDEMP', 'RCPPDEMP_F', 'RCPPDEMP_PCT', 'RCPPDEMP_PCT_F', 'EMP',
       'EMP_F', 'EMP_PCT', 'EMP_PCT_F', 'PAYANN', 'PAYANN_F', 'PAYANN_PCT',
       'PAYANN_PCT_F', 'QDESC_LABEL', 'us'],
      dtype='object', name=0)

In [60]:
df['BUSCHAR_LABEL'].value_counts()

All firms                                                                       1321
Total reporting                                                                 1307
Full-time paid employees                                                        1285
Part-time paid employees                                                        1266
Contractors, subcontractors, independent contractors, or outside consultants    1248
Item not reported                                                               1188
None of the above                                                               1008
Temporary staffing obtained from a temporary help service                        924
Paid day laborers                                                                917
Leased employees from leasing service or professional employer organization      769
Name: BUSCHAR_LABEL, dtype: int64

In [39]:
# Armin
# key = '7225d64f8070c982b47274ec2b7435f9b1addca9'

header = ['annual_payroll', 'industry', 'sex', 'ethnicity', 'race', 'buschar_label', 'number_of_firms', 'firmpdemp_f' ,'revenue', 'number_of_employees']
customers = to_dataframe(business_char_cust_us_request)
customers.to_csv('CSV/business_char_customers_us.csv', columns=header)


workers = to_dataframe(business_char_workers_us_request)
workers.to_csv('CSV/business_char_workers_us.csv', columns=header)


family = to_dataframe(business_char_family_us_request)
family.to_csv('CSV/business_char_family_us.csv', columns=header)



In [ ]:
# key = '7225d64f8070c982b47274ec2b7435f9b1addca9'
# business_char_family_us_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=us:*&QDESC_LABEL=FAMOWN&key={API_KEY}'

In [158]:
family = family[family.revenue != 'S']
family = family[family.revenue != 'W']
family['revenue'].value_counts()

family.head()


,geo_id,name,naics2017,industry,sex_code,sex,eth_group,ethnicity,race_group,race,vet_group,vet_group_label,qdesc,buschar,buschar_label,year,number_of_firms,firmpdemp_f,percent_of_firms,firmpdemp_pct_f,revenue,rcppdemp_f,percent_revenue_of_firms,rcppdemp_pct_f,number_of_employees,emp_f,percent_of_employees,emp_pct_f,annual_payroll,payann_f,percent_annual_payroll,payann_pct_f,qdesc_label,us
0,0100000US,United States,00,Total for all sectors,004,Equally male/female,028,Equally Hispanic/non-Hispanic,00,Total,001,Total,B02,BN,Not family-owned,2018,1725,NaN,10.2,NaN,1776514,NaN,11.2,NaN,14101,NaN,11.0,NaN,486843,NaN,12.1,NaN,FAMOWN,1
1,0100000US,United States,00,Total for all sectors,004,Equally male/female,028,Equally Hispanic/non-Hispanic,00,Total,001,Total,B02,BO,Total reporting,2018,16855,NaN,100.0,NaN,15807468,NaN,100.0,NaN,127908,NaN,100.0,NaN,4024678,NaN,100.0,NaN,FAMOWN,1
2,0100000US,United States,00,Total for all sectors,004,Equally male/female,028,Equally Hispanic/non-Hispanic,00,Total,001,Total,B02,BP,Item not reported,2018,658,NaN,0.0,X,546420,NaN,0.0,X,6487,NaN,0.0,X,150753,NaN,0.0,X,FAMOWN,1
3,0100000US,United States,00,Total for all sectors,001,Total,029,Non-Hispanic,00,Total,001,Total,B02,A1,All firms,2018,5118211,NaN,0.0,X,13830500008,NaN,0.0,X,60007007,NaN,0.0,X,2689941006,NaN,0.0,X,FAMOWN,1
4,0100000US,United States,00,Total for all sectors,001,Total,029,Non-Hispanic,00,Total,001,Total,B02,BM,Family-owned,2018,733366,NaN,29.2,NaN,1670082978,NaN,37.5,NaN,7552337,NaN,35.7,NaN,294296376,NaN,32.4,NaN,FAMOWN,1


In [59]:
customers = to_dataframe(business_char_cust_us_request)

customers = customers.groupby(['industry', 'race', 'buschar_label'])['number_of_firms'].count().unstack()

customers.head(20)


buschar_label                                                                                  All firms  \
industry                                           race                                                    
Accommodation and food services                    American Indian and Alaska Native                 1.0   
                                                   Asian                                             1.0   
                                                   Black or African American                         1.0   
                                                   Classifiable                                      1.0   
                                                   Equally minority/nonminority                      6.0   
                                                   Minority                                          7.0   
                                                   Native Hawaiian and Other Pacific Islander        1.0   
                                                   Nonminority                                       7.0   
                                                   Total                                            36.0   
                                                   Unclassifiable                                    1.0   
                                                   White                                             1.0   
Administrative and support and waste management... American Indian and Alaska Native                 1.0   
                                                   Asian                                             1.0   
                                                   Black or African American                         1.0   
                                                   Classifiable                                      1.0   
                                                   Equally minority/nonminority                      6.0   
                                                   Minority                                          7.0   
                                                   Native Hawaiian and Other Pacific Islander        1.0   
                                                   Nonminority                                       7.0   
                                                   Total                                            36.0   

buschar_label                                                                                  Federal government  \
industry                                           race                                                             
Accommodation and food services                    American Indian and Alaska Native                          1.0   
                                                   Asian                                                      1.0   
                                                   Black or African American                                  1.0   
                                                   Classifiable                                               1.0   
                                                   Equally minority/nonminority                               3.0   
                                                   Minority                                                   6.0   
                                                   Native Hawaiian and Other Pacific Islander                 NaN   
                                                   Nonminority                                                6.0   
                                                   Total                                                     21.0   
                                                   Unclassifiable                                             1.0   
                                                   White                                                      1.0   
Administrative and support and waste management... American Indian and Alaska Native                  